In [15]:
!pip install pyspark

In [16]:
from google.colab import drive
from pyspark.sql import SparkSession

In [17]:
drive.mount('/content/drive')
dados = '/content/drive/MyDrive/Workspace/fiap/pos-tech/data-analytics'
dados += '/fase-3/02-tech-challenge/dados'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
uf = {
    '11': 'Rondônia',
    '12': 'Acre',
    '13': 'Amazonas',
    '14': 'Roraima',
    '15': 'Pará',
    '16': 'Amapá',
    '17': 'Tocantins',
    '21': 'Maranhão',
    '22': 'Piauí',
    '23': 'Ceará',
    '24': 'Rio Grande do Norte',
    '25': 'Paraíba',
    '26': 'Pernambuco',
    '27': 'Alagoas',
    '28': 'Sergipe',
    '29': 'Bahia',
    '31': 'Minas Gerais',
    '32': 'Espírito Santo',
    '33': 'Rio de Janeiro',
    '35': 'São Paulo',
    '41': 'Paraná',
    '42': 'Santa Catarina',
    '43': 'Rio Grande do Sul',
    '50': 'Mato Grosso do Sul',
    '51': 'Mato Grosso',
    '52': 'Goiás',
    '53': 'Distrito Federal'
}

In [19]:
capital = {
    '11': 'Município de Porto Velho (RO)',
    '12': 'Município de Rio Branco (AC)',
    '13': 'Município de Manaus (AM)',
    '14': 'Município de Boa Vista (RR)',
    '15': 'Município de Belém (PA)',
    '16': 'Município de Macapá (AP)',
    '17': 'Município de Palmas (TO)',
    '21': 'Município de São Luís (MA)',
    '22': 'Município de Teresina (PI)',
    '23': 'Município de Fortaleza (CE)',
    '24': 'Município de Natal (RN)',
    '25': 'Município de João Pessoa (PB)',
    '26': 'Município de Recife (PE)',
    '27': 'Município de Maceió (AL)',
    '28': 'Município de Aracaju (SE)',
    '29': 'Município de Salvador (BA)',
    '31': 'Município de Belo Horizonte (MG)',
    '32': 'Município de Vitória (ES)',
    '33': 'Município de Rio de Janeiro (RJ)',
    '35': 'Município de São Paulo (SP)',
    '41': 'Município de Curitiba (PR)',
    '42': 'Município de Florianópolis (SC)',
    '43': 'Município de Porto Alegre (RS)',
    '50': 'Município de Campo Grande (MS)',
    '51': 'Município de Cuiabá (MT)',
    '52': 'Município de Goiânia (GO)',
    '53': 'Município de Brasília (DF)'
}

In [20]:
rm_ride = {
    '13': 'Região Metropolitana de Manaus (AM)',
    '15': 'Região Metropolitana de Belém (PA)',
    '16': 'Região Metropolitana de Macapá (AP)',
    '21': 'Região Metropolitana de Grande São Luís (MA)',
    '22': 'Região Administrativa Integrada de Desenvolvimento da Grande Teresina (PI)',
    '23': 'Região Metropolitana de Fortaleza (CE)',
    '24': 'Região Metropolitana de Natal (RN)',
    '25': 'Região Metropolitana de João Pessoa (PB)',
    '26': 'Região Metropolitana de Recife (PE)',
    '27': 'Região Metropolitana de Maceió (AL)',
    '28': 'Região Metropolitana de Aracaju (SE)',
    '29': 'Região Metropolitana de Salvador (BA)',
    '31': 'Região Metropolitana de Belo Horizonte (MG)',
    '32': 'Região Metropolitana de Grande Vitória (ES)',
    '33': 'Região Metropolitana de Rio de Janeiro (RJ)',
    '35': 'Região Metropolitana de São Paulo (SP)',
    '41': 'Região Metropolitana de Curitiba (PR)',
    '42': 'Região Metropolitana de Florianópolis (SC)',
    '43': 'Região Metropolitana de Porto Alegre (RS)',
    '51': 'Região Metropolitana de Vale do Rio Cuiabá (MT)',
    '52': 'Região Metropolitana de Goiânia (GO)'
}

In [21]:
v1022 = {
    '1': 'Urbana',
    '2': 'Rural'
}

In [22]:
v1023 = {
    '1': 'Capital',
    '2': 'Resto da RM (Região Metropolitana, excluindo a capital)',
    '3': 'Resto da RIDE (Região Integrada de Desenvolvimento Econômico, excluindo a capital)',
    '4': 'Resto da UF (Unidade da Federação, excluindo a região metropolitana e a RIDE)'
}

In [23]:
a001a = {
    '01': 'Pessoa responsável pelo domicílio',
    '02': 'Cônjuge ou companheiro(a) de sexo diferente',
    '03': 'Cônjuge ou companheiro(a) do mesmo sexo',
    '04': 'Filho(a) do responsável e do cônjuge',
    '05': 'Filho(a) somente do responsável',
    '06': 'Filho(a) somente do cônjuge',
    '07': 'Genro ou nora',
    '08': 'Pai, mãe, padrasto ou madrasta',
    '09': 'Sogro(a)',
    '10': 'Neto(a)',
    '11': 'Bisneto(a)',
    '12': 'Irmão ou irmã',
    '13': 'Avô ou avó',
    '14': 'Outro parente',
    '15': 'Agregado(a) - Não parente que não compartilha despesas',
    '16': 'Convivente - Não parente que compartilha despesas',
    '17': 'Pensionista',
    '18': 'Empregado(a) doméstico(a)',
    '19': 'Parente do(a) empregado(a) doméstico(a)'
}

In [24]:
a003 = {
    '1': 'Homem',
    '2': 'Mulher'
}

In [25]:
a004 = {
    '1': 'Branca',
    '2': 'Preta',
    '3': 'Amarela',
    '4': 'Parda',
    '5': 'Indígena',
    '9': 'Ignorado'
}

In [26]:
a005 = {
    '1': 'Sem instrução',
    '2': 'Fundamental incompleto',
    '3': 'Fundamental completa',
    '4': 'Médio incompleto',
    '5': 'Médio completo',
    '6': 'Superior incompleto',
    '7': 'Superior completo',
    '8': 'Pós-graduação, mestrado ou doutorado'
}

In [27]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [28]:
spark.read.csv(f'{dados}/pnad-covid-2020-*.csv.gz', header=True).show()

+----+---+-------+-------+-----+-----+-----+-----+-------+---------+-----+-----+------+------------+------------+------+----+-----+------+------+------+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+------+------+-----+----+----+-----+----+-----+------+--------+-----+------+------+-----+-----+-----+------+-------+--------+------+-------+-------+----+----+----+----+----+-----+-----+--------+-----+-----+-----+-----+-----+--------+-----+--------+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+------+------+------+------+------+-----+----+
| Ano| UF|CAPITAL|RM_RIDE|V1008|V1012|V1013|V1016|Estrato|      UPA|V1022|V1023| V1030|       V103